# using gattlib

In [1]:
import sys
from gattlib import GATTRequester
st_addres= "D5:C9:57:C2:25:7B"

In [2]:
requester = GATTRequester(st_addres, False)
requester.connect(True, channel_type="random")

In [3]:
requester.discover_primary()

[{'uuid': '00001801-0000-1000-8000-00805f9b34fb', 'start': 1, 'end': 4},
 {'uuid': '00001800-0000-1000-8000-00805f9b34fb', 'start': 5, 'end': 11},
 {'uuid': '00000000-0001-11e1-9ab4-0002a5d5c51b', 'start': 12, 'end': 18},
 {'uuid': '00000000-0002-11e1-9ab4-0002a5d5c51b', 'start': 28, 'end': 31},
 {'uuid': '00000000-0003-11e1-9ab4-0002a5d5c51b', 'start': 32, 'end': 35}]

In [4]:
characteristics = requester.discover_characteristics()
characteristics

[{'uuid': '00002a05-0000-1000-8000-00805f9b34fb',
  'handle': 2,
  'properties': 32,
  'value_handle': 3},
 {'uuid': '00002a00-0000-1000-8000-00805f9b34fb',
  'handle': 6,
  'properties': 78,
  'value_handle': 7},
 {'uuid': '00002a01-0000-1000-8000-00805f9b34fb',
  'handle': 8,
  'properties': 78,
  'value_handle': 9},
 {'uuid': '00002a04-0000-1000-8000-00805f9b34fb',
  'handle': 10,
  'properties': 2,
  'value_handle': 11},
 {'uuid': '00140000-0001-11e1-ac36-0002a5d5c51b',
  'handle': 13,
  'properties': 18,
  'value_handle': 14},
 {'uuid': '00e00000-0001-11e1-ac36-0002a5d5c51b',
  'handle': 16,
  'properties': 16,
  'value_handle': 17},
 {'uuid': '00000100-0001-11e1-ac36-0002a5d5c51b',
  'handle': 29,
  'properties': 16,
  'value_handle': 30},
 {'uuid': '20000000-0001-11e1-ac36-0002a5d5c51b',
  'handle': 33,
  'properties': 22,
  'value_handle': 34}]

![alt text](Properties.png "Title")

In [5]:
def gen(x):
    while x:
        yield bin(x % 16)[2:].zfill(4)
        x >>= 4

def nibble(x):
    return '0b'+'_'.join(list(gen(x))[::-1])

In [6]:
requester.read_by_handle(34)

[b'\x81\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [7]:
import time
start = time.time()
for _ in range(10):
    requester.read_by_handle(34)
end = time.time()
print((end - start)/10.0)

1.1269385099411011


In [ ]:
def print_handle_14(d):
    print('d:', [hex(di) for di in d])
    tick = int.from_bytes(d[:2], "little")
    print("tick:",tick)
    pressure = float(int.from_bytes(d[2:6], "little"))/100
    print("pressure:",pressure)
    temprature = float(int.from_bytes(d[6:], "little"))/10.0
    print("temprature:",temprature)

In [ ]:
# read data diraectly
data = requester.read_by_handle(0x000e)
print_handle_14(data[0])

In [ ]:
# start notification
requester.write_by_handle(15, b"\x01\x00")

In [ ]:
# stop notification
requester.write_by_handle(15, b"\x00\x00")

In [ ]:
requester.disconnect()
del requester

## using notifications to read data

In [ ]:
# read using notification on handle 0x0e (or 14)
from threading import Event


class Requester(GATTRequester):
    def __init__(self, wakeup, *args):
        GATTRequester.__init__(self, *args)
        self.wakeup = wakeup
        self.counter = 0

    def on_notification(self, handle, data):
        print("-------------------------------------------")
        print("- notification on handle: {}".format(handle))
        print(data)
        print_handle_14(data[-8:])
        # wait for 10 notifications
        self.counter = self.counter + 1
        if self.counter > 10:
            self.wakeup.set()


class ReceiveNotification(object):
    def __init__(self, address):
        self.received = Event()
        self.requester = Requester(self.received, address, False)
        self.connect()
    
    def main(self):
        self.wait_notification()

    def connect(self):
        print("Connecting...", end=' ')
        sys.stdout.flush()

        self.requester.connect(True, channel_type="random")
        print("OK!")

    def wait_notification(self):
        self.requester.write_by_handle(14+1, b"\x01\x00")
        print("\nThis is a bit tricky. You need to make your device to send\n"
              "some notification. I'll wait...")
        self.received.wait()
        # turn off notification
        self.requester.write_by_handle(14+1, b"\x00\x00")

p = ReceiveNotification(st_addres)
p.main()

In [ ]:
# p.requester.write_by_handle(14 + 1, b"\x00\x00")
p.requester.disconnect()

# using bluepy

In [ ]:
from bluepy import btle
st_addres = "D5:C9:57:C2:25:7B"
p = btle.Peripheral(st_addres, btle.ADDR_TYPE_RANDOM)

In [ ]:
class MyDelegate(btle.DefaultDelegate):
    def __init__(self):
        btle.DefaultDelegate.__init__(self)

    def handleNotification(self, cHandle, data):
        print("A notification was received: {}".format(data))


p.setDelegate( MyDelegate() )

In [ ]:
# Setup to turn notifications on, e.g.
svc = p.getServiceByUUID('00000000-0002-11e1-9ab4-0002a5d5c51b')
ch = svc.getCharacteristics()[0]

# enable notification
p.writeCharacteristic(ch.valHandle+1, b"\x01\x00")

counter = 0

while True:
    if p.waitForNotifications(1.0):
        counter += 1
        # wait for five notification
        if counter > 5:
            break

# disable notification
p.writeCharacteristic(ch.valHandle+1, b"\x00\x00")

In [ ]:
char = p.getCharacteristics(uuid = '00e00000-0001-11e1-ac36-0002a5d5c51b')[0]
char.propertiesToString()
char.getHandle()

In [ ]:
srvs = p.getServices()
srvs = list(srvs)
[srv.uuid.getCommonName() for srv in srvs]

In [ ]:
chars = p.getCharacteristics()
chars = list(chars)
[srv.uuid.getCommonName() for srv in srvs]

In [ ]:
p.disconnect()

# L2CAP

In [ ]:
import bluetooth

In [ ]:
import bluetooth
sock=bluetooth.BluetoothSocket( bluetooth.L2CAP )
st_addres = "D5:C9:57:C2:25:7B"
port = 0x1001
sock.connect((st_addres, port))


In [ ]:
del sock